# Model for Nature Conservancy Fisheries Kaggle Competition

#### Dependencies

In [1]:
import fish_data as fd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import os

#### Helper functions

In [2]:
help(fd)

Help on module fish_data:

NAME
    fish_data

DESCRIPTION
    fish_data module contains the helper functions for the model build of the
    Nature Conservancy Fisheries Kaggle Competition.
    
    Dependencies:
        * numpy as np
        * os
        * scipy.ndimage as ndimage
        * scipy.misc as misc
        * scipy.special as special
        * matplotlib.pyplot as plt
        * tensorflow as tf

FUNCTIONS
    count_nodes(y_in, x_in, conv_depths, conv_strides, pool_strides)
        Calculates the number of total nodes present in the last layer of a
        convolution plus max_pooling architecture.  Calculations assume that
        convolution is 'SAME' padded, and pooling is 'VALID' padded.
    
    decode_image(image_read, size, num_channels=3, mutate=False, brightness_delta=None, contrast_limits=None, hue_delta=None, saturation_limits=None)
        Converts a dequeued image read from filename to a single tensor array,
        with modifications:
            * resized to st

#### Generate a list of filenames

In [3]:
fish_filenames = fd.generate_filenames_list()
print("There are {} filenames in the master set list".format(len(fish_filenames)))

There are 3777 filenames in the master set list


In [4]:
fish_label_arr = fd.make_labels(fish_filenames, 'train/', '/img')
fish_label_arr.shape
fish_label_arr[0:5,:]

array([[1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0]])

In [5]:
valid_size = 75
files_train, files_val, y_train, y_val = train_test_split(fish_filenames, fish_label_arr, test_size = valid_size)
print([x.shape for x in [y_train, y_val]])

[(3702, 8), (75, 8)]


#### Debugging Graph and session calls with input pipeline

## Graph and Session Runs

#### Graph parameters

In [6]:
#Preprocessing
std_y = 300
std_x = 500

# General
num_channels = 3
num_labels = 8
batch_size = 25
stddev = 0.2

# convolution
kernel_sizes = [12, 3, 3, 3, 3, 3]
conv_depths = [64, 128, 256, 512, 256, 128]
conv_strides = [4, 1, 1, 1, 1, 1]

pool_strides = [2, 2, 2, 2]

final_depth = conv_depths[-1]

#dropout
kp = 0.75

# fully connected
fc1_depth = 256
fc2_depth = 64

#regularization
beta = 1e-1 

# Learning rate
init_rate = 5e-3
per_steps = 6000
decay_rate = 0.75


#### Session parameters

In [7]:
# epochs
num_epochs = 5
# path for tensorboard summary file to be written
logs_path = os.getcwd()+'/TB_logs'
valid_every = 25

In [11]:
%run -i 'GRAPH.py'

In [13]:
%run -i 'SESSION.py'

Initialized!


To view your tensorboard dashboard summary, run the following on the command line:
tensorboard --logdir='/Users/ccthomps/Documents/Python Files/Kaggle Competitions/Nature Conservancy Fisheries/TB_logs'
Train_Mean_Cross_entropy: 543133.6875
     Valid_mean_cross_entropy: 338082.5625
Train_Mean_Cross_entropy: 350593.875
Train_Mean_Cross_entropy: 90598.796875
Train_Mean_Cross_entropy: 32313.025390625
Train_Mean_Cross_entropy: 53913.84375
Train_Mean_Cross_entropy: 33691.0234375
Train_Mean_Cross_entropy: 8119.458984375
Train_Mean_Cross_entropy: 15893.068359375
Train_Mean_Cross_entropy: 12028.447265625
Train_Mean_Cross_entropy: 9965.16796875
Train_Mean_Cross_entropy: 13634.5986328125
     Valid_mean_cross_entropy: 201049.359375
Train_Mean_Cross_entropy: 11598.40625
Train_Mean_Cross_entropy: 6139.25927734375
Train_Mean_Cross_entropy: 9656.85546875
Train_Mean_Cross_entropy: 10683.2841796875
Train_Mean_Cross_entropy: 7661.1298828125
Train_Mean_Cross_entropy: 6785.009765625
Train_